# Preparing Data for Text Processing

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [3]:
df.shape

(50000, 2)

# Bag-of-Words Model

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

# initialize and fit bag of words model
count = CountVectorizer()
docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining, the weather is sweet, '
    'and one and one is two'])
bag = count.fit_transform(docs)

In [5]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [6]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [7]:
from sklearn.feature_extraction.text import TfidfTransformer

# transform raw term frequencies into tf-idfs
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [8]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [9]:
import re

def preprocessor(text):
    """Remove HTML markup and move emoticons to end of string."""
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) +  ' '.join(emoticons).replace('-', ''))
    return text

In [10]:
print(preprocessor(df.loc[0, 'review'][-50:]))
print(preprocessor("</a>This :) is :( a test :-)!"))

is seven title brazil not available
this is a test :) :( :)


In [11]:
df['review'] = df['review'].apply(preprocessor)

In [12]:
def tokenizer(text):
    """Split documents into individual words."""
    return text.split()

tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [13]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    """Reduce words to root form."""
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [14]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/kalyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
from nltk.corpus import stopwords

# remove english stopwords
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

# Logistic Regression for Document Classification

In [16]:
# split dataset into features and target training and test sets
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)
tfidf_X_train = tfidf.fit_transform(X_train)

clf = LogisticRegression(solver='lbfgs', random_state=0)
clf.fit(tfidf_X_train, y_train)
tfidf_X_test = tfidf.transform(X_test)
print(tfidf_X_test.shape)

(25000, 78041)


In [18]:
scores = cross_val_score(clf, tfidf_X_test, y_test, cv=5)
acc = scores.mean()
print(f"Accuracy: {(acc *100):.2f} percent")

Accuracy: 88.51 percent


# Working with Big Data  
We can make the whole dataset into batches and run the desired algorithm 

In [19]:
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [20]:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # skip header
        for line in csv:
            text, label = line[: -3], int(line[-2])
            yield text, label

In [21]:
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [22]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [23]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

# Initialize HashingVectorizer
vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)
# Initialize SGDclassifier
clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
# Return docs
doc_stream = stream_docs(path='movie_data.csv')

In [24]:


classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)

/home/kalyan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


In [25]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print(f'Accuracy: {clf.score(X_test, y_test):.3f}')

Accuracy: 0.867


In [26]:
clf = clf.partial_fit(X_test, y_test)
clf

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=1, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

# Topic Modeling with Latent Dirichlet Allocation

In [27]:
count = CountVectorizer(stop_words='english', max_df=.1, max_features=2000)
X = count.fit_transform(df['review'].values)

In [28]:
from sklearn.decomposition import LatentDirichletAllocation

# Initialize and fit LDA
lda = LatentDirichletAllocation(n_components=10, random_state=123, learning_method='batch')
X_topics = lda.fit_transform(X)

In [29]:
lda.components_.shape

(10, 2000)

In [30]:
n_top_words = 5
feature_names = count.get_feature_names()

for topic_idx, topic in enumerate(lda.components_):
    print(f'Topic {(topic_idx + 1)}')
    print(" ".join(
        [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    ))

Topic 1
book performance role version wonderful
Topic 2
worst minutes boring script terrible
Topic 3
action black white sex budget
Topic 4
john role wife plays murder
Topic 5
horror comedy kids children family
Topic 6
family mother father woman girl
Topic 7
war american documentary human cinema
Topic 8
series episode tv effects episodes
Topic 9
comedy guy jokes guys laugh
Topic 10
dvd video music worst minutes


In [31]:
horror = X_topics[:, 4].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print(f'\nHorror movie {(iter_idx + 1)}')
    print(df['review'][movie_idx][:300], '...')


Horror movie 1
perhaps one of the first slasher film that came out after halloween although made from irwin yabalans from halloween i must say i honestly found tourist trap to be scarier and funner tourist trap is one of those remarkable treats you find every now and then and are left with the most enjoyable feeli ...

Horror movie 2
night of the demons is definitely one the definitive cheesy 80 s demons horror flick in the same vein as the brilliant evil dead and demons movies this movie combines boozy sexually active teens and demons into one hell of a fun movie a definite welcome addition after the 80 s were overrun by slashe ...

Horror movie 3
okay pretty good doesn t scream rent me but i was surprised at how much i enjoyed campfire tales while by no means a classic the directors involved do have an idea what suspense is the scares don t just jump out at you but instead the directors build steadily to the climatic moments the film is base ...
